# SkillFactory
## Введение в ML, введение в sklearn

В этом задании мы с вами рассмотрим данные с конкурса [Задача предсказания отклика клиентов ОТП Банка](http://www.machinelearning.ru/wiki/index.php?title=%D0%97%D0%B0%D0%B4%D0%B0%D1%87%D0%B0_%D0%BF%D1%80%D0%B5%D0%B4%D1%81%D0%BA%D0%B0%D0%B7%D0%B0%D0%BD%D0%B8%D1%8F_%D0%BE%D1%82%D0%BA%D0%BB%D0%B8%D0%BA%D0%B0_%D0%BA%D0%BB%D0%B8%D0%B5%D0%BD%D1%82%D0%BE%D0%B2_%D0%9E%D0%A2%D0%9F_%D0%91%D0%B0%D0%BD%D0%BA%D0%B0_%28%D0%BA%D0%BE%D0%BD%D0%BA%D1%83%D1%80%D1%81%29)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12,5)

### Грузим данные

Считаем описание данных

In [3]:

df_descr = pd.read_csv('data/otp_description.csv', sep='\t', encoding='utf8')

In [4]:
for i in range(df_descr.shape[0]):
    print(df_descr.iloc[i,0], "-", df_descr.iloc[i,1])

AGREEMENT_RK - уникальный идентификатор объекта в выборке
AGE - возраст клиента
SOCSTATUS_WORK_FL - социальный статус клиента относительно работы (1 - работает, 0 - не работает)
SOCSTATUS_PENS_FL - социальный статус клиента относительно пенсии (1 - пенсионер, 0 - не пенсионер)
GENDER - пол клиента (1- женщины, 0-мужчины)
CHILD_TOTAL - количество детей клиента
DEPENDANTS - количество иждивенцев клиента
EDUCATION - образование
MARITAL_STATUS - семейное положение
GEN_INDUSTRY - отрасль работы клиента
GEN_TITLE - должность
ORG_TP_STATE - форма собственности компании
ORG_TP_FCAPITAL - отношение к иностранному капиталу
JOB_DIR - направление деятельности в нутри компании
FAMILY_INCOME - семейный доход (несколько категорий)
PERSONAL_INCOME - личный доход клиента (в рублях)
REG_ADDRESS_PROVINCE - область регистрации клиента
FACT_ADDRESS_PROVINCE - область фактического пребывания клиента
POSTAL_ADDRESS_PROVINCE - почтовый адрес область
TP_PROVINCE - область торговой точки, где клиент брал послед

Считаем обучающую выборки и тестовую (которую мы как бы не видим)

In [5]:
df_train = pd.read_csv('data/otp_train.csv', sep='\t', encoding='utf8')

In [6]:
df_train.shape

(15223, 52)

In [7]:
df_test = pd.read_csv('data/otp_test.csv', sep='\t', encoding='utf8')

In [8]:
df_test.shape

(14910, 52)

In [9]:
df_train.head()

,AGREEMENT_RK,TARGET,AGE,SOCSTATUS_WORK_FL,SOCSTATUS_PENS_FL,GENDER,CHILD_TOTAL,DEPENDANTS,EDUCATION,MARITAL_STATUS,...,REG_PHONE_FL,GEN_PHONE_FL,LOAN_NUM_TOTAL,LOAN_NUM_CLOSED,LOAN_NUM_PAYM,LOAN_DLQ_NUM,LOAN_MAX_DLQ,LOAN_AVG_DLQ_AMT,LOAN_MAX_DLQ_AMT,PREVIOUS_CARD_NUM_UTILIZED
0,59910150,0,49,1,0,1,2,1,Среднее специальное,Состою в браке,...,0,1,1,1,6,2,1,1580,1580,NaN
1,59910230,0,32,1,0,1,3,3,Среднее,Состою в браке,...,0,1,1,1,6,1,1,4020,4020,NaN
2,59910525,0,52,1,0,1,4,0,Неполное среднее,Состою в браке,...,0,1,2,1,11,0,0,0,0,NaN
3,59910803,0,39,1,0,1,1,1,Высшее,Состою в браке,...,1,1,1,1,6,3,1,"1589,92333333333",1590,NaN
4,59911781,0,30,1,0,0,0,0,Среднее,Состою в браке,...,0,1,2,1,16,2,1,"1152,15",2230,NaN


## Объединим две выборки

Так как пока мы пока не умеем работать sklearn  Pipeline, то для того, чтобы после предобработки столбцы в двух выборках находились на своих местах.

Для того, чтобы в дальнейшем отделить их введем новый столбец "sample"

In [10]:
df_train.loc[:, 'sample'] = 'train'
df_test.loc[:, 'sample'] = 'test'

In [11]:
df = df_test.append(df_train).reset_index(drop=True)

In [12]:
df.shape

(30133, 53)

### Чуть-чуть посмотрим на данные

Посмотрим типы данных и их заполняемость

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30133 entries, 0 to 30132
Data columns (total 53 columns):
AGREEMENT_RK                  30133 non-null int64
TARGET                        30133 non-null int64
AGE                           30133 non-null int64
SOCSTATUS_WORK_FL             30133 non-null int64
SOCSTATUS_PENS_FL             30133 non-null int64
GENDER                        30133 non-null int64
CHILD_TOTAL                   30133 non-null int64
DEPENDANTS                    30133 non-null int64
EDUCATION                     30133 non-null object
MARITAL_STATUS                30133 non-null object
GEN_INDUSTRY                  27420 non-null object
GEN_TITLE                     27420 non-null object
ORG_TP_STATE                  27420 non-null object
ORG_TP_FCAPITAL               27425 non-null object
JOB_DIR                       27420 non-null object
FAMILY_INCOME                 30133 non-null object
PERSONAL_INCOME               30133 non-null object
REG_ADDRESS_PRO

Видим, что часть данных - object, скорее всего стоки.


Давайте выведем эти значения для каждого столбца

In [14]:
for i in df_train.columns: # перебираем все столбцы
    if str(df_train[i].dtype) == 'object': # если тип столбца - object
        print('='*10)
        print(i) # выводим название столбца
        print(set(df_train[i])) # выводим все его значения (но делаем set - чтоб значения не повторялись)
        print('\n') # выводим пустую строку

EDUCATION
{'Неоконченное высшее', 'Среднее', 'Ученая степень', 'Среднее специальное', 'Неполное среднее', 'Высшее', 'Два и более высших образования'}


MARITAL_STATUS
{'Состою в браке', 'Вдовец/Вдова', 'Разведен(а)', 'Не состоял в браке', 'Гражданский брак'}


GEN_INDUSTRY
{nan, 'Торговля', 'Энергетика', 'Образование', 'Управляющая компания', 'Маркетинг', 'Здравоохранение', 'Страхование', 'Наука', 'Коммунальное хоз-во/Дорожные службы', 'Сборочные производства', 'Химия/Парфюмерия/Фармацевтика', 'Информационные услуги', 'Информационные технологии', 'Банк/Финансы', 'Юридические услуги/нотариальные услуги', 'Салоны красоты и здоровья', 'Нефтегазовая промышленность', 'Недвижимость', 'Металлургия/Промышленность/Машиностроение', 'Сельское хозяйство', 'Логистика', 'Государственная служба', 'Транспорт', 'Другие сферы', 'Туризм', 'Развлечения/Искусство', 'Подбор персонала', 'Ресторанный бизнес/Общественное питание', 'Строительство', 'СМИ/Реклама/PR-агенства', 'ЧОП/Детективная д-ть'}


GEN_TITLE


Mожно заметить что некоторые переменные, которые обозначены как строки (например PERSONAL_INCOME) на самом деле числа, но по какой-то причине были распознаны как строки

Причина же что использовалась запятая для разделения не целой части числа..

Перекодировать их можно например так:

In [15]:
df['PERSONAL_INCOME'].map(lambda x: x.replace(',', '.')).astype('float')

0         7000.0
1         4100.0
2        40000.0
3        20000.0
4        17000.0
5         7000.0
6        10000.0
7         7500.0
8         9000.0
9         4000.0
10       15000.0
11       12000.0
12        8000.0
13       20000.0
14       18500.0
15       14000.0
16        6000.0
17        9000.0
18        5000.0
19       15000.0
20        6100.0
21       22180.0
22        9000.0
23        8700.0
24        7000.0
25        6000.0
26       15000.0
27       12000.0
28       11000.0
29        8000.0
          ...   
30103    12000.0
30104    10000.0
30105    10000.0
30106     8000.0
30107    11000.0
30108     7000.0
30109    10000.0
30110     8000.0
30111    25000.0
30112    10000.0
30113    12000.0
30114     7000.0
30115    12000.0
30116    14500.0
30117    12000.0
30118     8000.0
30119     7000.0
30120    10000.0
30121    10000.0
30122    10000.0
30123    15000.0
30124     8100.0
30125     9500.0
30126    13000.0
30127    10000.0
30128    12000.0
30129    10000.0
30130     6000

Такой эффект наблюдается в столбцах `PERSONAL_INCOME`, `CREDIT`, `FST_PAYMENT`, `LOAN_AVG_DLQ_AMT`, `LOAN_MAX_DLQ_AMT`

### Теперь ваше небольшое исследование

#### Задание 1. Есть ли пропуски в данных? Что с ними сделать?

(единственного верного ответа нет - аргументируйте)

In [16]:
#посчитаем пропуски в данных:
with_null = df.isnull().sum(axis = 0)
with_null[with_null != 0]

GEN_INDUSTRY                   2713
GEN_TITLE                      2713
ORG_TP_STATE                   2713
ORG_TP_FCAPITAL                2708
JOB_DIR                        2713
TP_PROVINCE                     590
REGION_NM                         2
WORK_TIME                      2717
PREVIOUS_CARD_NUM_UTILIZED    29533
dtype: int64

_решение:_ пропуски в данных есть, но способы их заполнения могут зависеть от причин.
для этого рассмотрим пропуски по столбцам отдельно
<br><br>
На сайте сказано, что данные пустые, так как клиенты не заполнили эти поля, но в любом случае нужно проверить на зависимость от других параметром

#### a) Факторы с одинаковым количеством пропусков

У факторов 'GEN_INDUSTRY', 'GEN_TITLE', 'ORG_TP_STATE', 'JOB_DIR' одинаковое количество пропусков. Проверим, находятся ли пропуски в одних и тех же строках. Например, есть ли в этих факторах не пустые значения, если значение 'GEN_INDUSTRY' пустое

In [17]:
df[df['GEN_INDUSTRY'].isnull() == True][['GEN_INDUSTRY',
                                         'GEN_TITLE',
                                         'ORG_TP_STATE',
                                         'JOB_DIR']].isnull().sum(axis = 0)

GEN_INDUSTRY    2713
GEN_TITLE       2713
ORG_TP_STATE    2713
JOB_DIR         2713
dtype: int64

In [18]:
#создадим отдельный df и посмотрим его подробнее
simult_na = df[df['GEN_INDUSTRY'].isnull() == True]
simult_na.head(n = 15).T

,1,5,8,12,20,28,30,57,77,96,103,111,156,177,192
AGREEMENT_RK,59910420,59911769,59912422,59912574,59913432,59914189,59914517,59918446,59920591,59923652,59924499,59924977,59930509,59932751,59935061
TARGET,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
AGE,48,44,53,59,65,63,63,62,58,66,59,61,55,59,63
SOCSTATUS_WORK_FL,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
SOCSTATUS_PENS_FL,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
GENDER,1,0,1,1,1,0,1,1,1,1,1,1,1,1,1
CHILD_TOTAL,0,2,2,0,2,0,2,0,2,0,0,0,4,2,2
DEPENDANTS,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0
EDUCATION,Среднее,Среднее специальное,Среднее специальное,Неполное среднее,Среднее специальное,Среднее специальное,Среднее специальное,Среднее специальное,Среднее специальное,Неполное среднее,Среднее специальное,Среднее специальное,Среднее,Среднее,Неполное среднее
MARITAL_STATUS,Состою в браке,Состою в браке,Состою в браке,Не состоял в браке,Разведен(а),Состою в браке,Не состоял в браке,Состою в браке,Вдовец/Вдова,Состою в браке,Не состоял в браке,Разведен(а),Не состоял в браке,Состою в браке,Разведен(а)


In [19]:
#скорее всего, данные не заполнены для всех неработающих и/или пенсионеров.
#посчитаем к-во записей по столбцам SOCSTATUS_WORK_FL и SOCSTATUS_PENS_FL в
#этом же df

p = simult_na.pivot_table(
    'TARGET', 'SOCSTATUS_WORK_FL', 'SOCSTATUS_PENS_FL', 'count')
p
#получается, что проблема была в отстутствии данных о работе у неработающих
#пенсионеров. При этом безработные в выборке отсутствуют
#также данных по перечисленным факторам нет для одного 24-летнего.
#Предположим, что это ошибка в данных, тогда пропуск можно заполнить
#самыми частыми значениями для его возрастной группы

SOCSTATUS_PENS_FL,0,1
SOCSTATUS_WORK_FL,,
0,NaN,2712.0
1,1.0,NaN


#### b) Фактор worktime

Похоже, пропуски в worktime тоже связаны с неработающими пенсионерами из предыдущего пункта. Однако пропусков там чуть больше, найдем в чем разница

In [22]:
no_worktime = df[df['WORK_TIME'].isnull() == True]
o_worktime = pd.merge(no_worktime, simult_na, how='outer')
i_worktime = pd.merge(no_worktime, simult_na)
set_diff_worktime = pd.concat([o_worktime, i_worktime]).drop_duplicates(keep=False)
#set_diff_worktime.T #в df 4 записи
#есть 4 человека со статусом безработный пенсионер, у которых есть данные о работодателе
#при этом не заполнены данные по времени работы в текущем месте. в остальном
#этот фактор совпадает с 4 предыдущими

#### c) Фактор ORG_TP_FCAPITAL

In [23]:
#скорее всего, пропуски в этом факторе тоже связаны с отсутствием данных по безработным пенсионерам
#поэтому найдем разницу между двумя df с пропусками в этих факторах

no_capital = df[df['ORG_TP_FCAPITAL'].isnull() == True]
o_capital = pd.merge(no_capital, simult_na, how='outer')
i_capital = pd.merge(no_capital, simult_na)
set_diff_capital = pd.concat([o_capital, i_capital]).drop_duplicates(keep=False)
#set_diff_capital.T #7 записей в df

Два множества отличаются на 7 записей. Будем считать, что причина одинаковая

#### d) Фактор TP_PROVINCE

TP_PROVINCE = область торговой точки, где клиент брал последний кредит
из описания данных на сайте не совсем понятно, какие клиенты "лежат" в датасете. Значит ли это, что клиент не брал кредит или что данных нет по какой-то другой причине (например, кредит был получен не в магазине)?

In [24]:
#проверим есть ли нули в поле CREDIT - это может означать, что в выборке есть клиенты без кредитов,
#тогда для них не будет и торговой точки выдачи кредита.
df[df['CREDIT'] == 0]

#в поле с суммами выданных кредитов нет пропусков и нулей

,AGREEMENT_RK,TARGET,AGE,SOCSTATUS_WORK_FL,SOCSTATUS_PENS_FL,GENDER,CHILD_TOTAL,DEPENDANTS,EDUCATION,MARITAL_STATUS,...,GEN_PHONE_FL,LOAN_NUM_TOTAL,LOAN_NUM_CLOSED,LOAN_NUM_PAYM,LOAN_DLQ_NUM,LOAN_MAX_DLQ,LOAN_AVG_DLQ_AMT,LOAN_MAX_DLQ_AMT,PREVIOUS_CARD_NUM_UTILIZED,sample


In [31]:
#в датафрейме кроме TP_PROVINCE есть еще 4 фактора, относящихся к географии,
#и 4 фактора, указывающие на совпадение этих измерений (среди них есть и
#совпадение по отсутствующему полю)
#посмотрим этот набор более подробно:

geo_col_list = ['REG_ADDRESS_PROVINCE', 'FACT_ADDRESS_PROVINCE', 'POSTAL_ADDRESS_PROVINCE',
               'TP_PROVINCE', 'REGION_NM', 'REG_FACT_FL', 'REG_POST_FL', 'REG_FACT_POST_FL',
               'REG_FACT_POST_TP_FL']

geo_df = df[geo_col_list]
geo_df.head()

,REG_ADDRESS_PROVINCE,FACT_ADDRESS_PROVINCE,POSTAL_ADDRESS_PROVINCE,TP_PROVINCE,REGION_NM,REG_FACT_FL,REG_POST_FL,REG_FACT_POST_FL,REG_FACT_POST_TP_FL
0,Красноярский край,Красноярский край,Красноярский край,Красноярский край,ВОСТОЧНО-СИБИРСКИЙ,0,0,0,1
1,Орловская область,Орловская область,Орловская область,Орловская область,ЦЕНТРАЛЬНЫЙ 1,1,1,1,1
2,Коми,Коми,Коми,Коми,ЦЕНТРАЛЬНЫЙ 2,1,1,1,1
3,Читинская область,Читинская область,Читинская область,Читинская область,ВОСТОЧНО-СИБИРСКИЙ,1,1,1,1
4,Хабаровский край,Хабаровский край,Хабаровский край,Хабаровский край,ДАЛЬНЕВОСТОЧНЫЙ,1,1,1,1


In [35]:
#отдельно посмотрим географические факторы с пропусками в TP_PROVINCE
geo_df = geo_df[geo_df['TP_PROVINCE'].isnull() == True]
geo_df

In [37]:
#отдельно посмотрим есть ли в логической переменной REG_FACT_POST_TP_FL для TP_PROVINCE единицы

geo_df[geo_df['REG_FACT_POST_TP_FL'] == 1]

,REG_ADDRESS_PROVINCE,FACT_ADDRESS_PROVINCE,POSTAL_ADDRESS_PROVINCE,TP_PROVINCE,REGION_NM,REG_FACT_FL,REG_POST_FL,REG_FACT_POST_FL,REG_FACT_POST_TP_FL


*Ответ на задание 1:* <br>
пропуски есть. основные группы/факторы/причины:<br>
1. __**проблема:**__ есть пропуски для неработающих пенсионеров в 6 столбцах (и совсем небольшое число пропусков там-же, которые можно списать на ошибки при сборе данных<br>
__**решение:**__ можно сделать отдельный "класс" - неработающий пенсионер
2. __**проблема:**__ 590 пропусков TP_PROVINCE - причина пока не ясна. <br>__**решение:**__ Но есть много других географических данных, можно заполнить данными из одного из них для каждого конкретного клиента. Делаем допущение, что часто они совпадают (нужно проверить!!)
3. __**проблема:**__ PREVIOUS_CARD_NUM_UTILIZED <br>__**решение:**__ согласно описанию данных: пропуск = 0. просто заполним

#### Задание 2. Есть ли категориальные признаки? Что с ними делать?

#### Задание 3. Фунция предобработки

Напишите функцию, которая бы

* Удаляло идентификатор `AGREEMENT_RK`
* Избавлялась от проблем с '.' и ',' в стобцах PERSONAL_INCOME, CREDIT, FST_PAYMENT, LOAN_AVG_DLQ_AMT, LOAN_MAX_DLQ_AMT
* Что-то делала с пропусками
* Кодировала категориальные признаки

В результате, ваш датафрейм должен содержать только числа и не содержать пропусков!

In [ ]:
def preproc_data(df_input):
    df_output = df_input.copy()
    
    
    ## Your Code Here
    
    return df_output

In [ ]:
df_preproc = df.pipe(preproc_data)

df_train_preproc = df_preproc.query('sample == "train"').drop(['sample'], axis=1)
df_test_preproc = df_preproc.query('sample == "test"').drop(['sample'], axis=1)

#### Задание 4. Отделите целевую переменную и остальные признаки

Должно получится:
* 2 матрицы: X и X_test
* 2 вектора: y и y_test

#### Задание 5. Обучение и оценка качества разных моделей

In [ ]:
from sklearn.cross_validation import train_test_split
# test_size=0.3, random_state=42

## Your Code Here


In [ ]:
# Попробовать следующие "черные ящики": интерфейс одинаковый 
# Постепенно мы узнаем, что они делают а сейчас учимся понимать какой работает качественнее
#     fit, 
#     predict, 
#     predict_proba

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

## Your Code Here

In [ ]:
# Посчитать метрики стандартные
# accuracy, precision, recall

from sklearn.metrics import accuracy_score, precision_score, recall_score

## Your Code Here

In [ ]:
# Визуалищировать эти метрики всех моделей на одном графике (чтоб визуально сравнить)
# Возможно вас удивит качество! Но задача подобрана специально ;) Такое качество тоже бывает

## Your Code Here

In [ ]:
# Потроить roc-кривые всех можелей на одном графике
# Вывести roc_auc каждой моделе
# Подпишите оси и линии

## Your Code Here

In [ ]:
from sklearn.cross_validation import cross_val_score
# Сделать k-fold (10 фолдов) кросс-валидацию каждой модели
# И посчитать средний roc_auc

## Your Code Here

In [ ]:
# Взять лучшую модель и сделать предсказания (с вероятностями (!!!)) для test выборки

predict = 
## Your Code Here

In [ ]:
# Померить roc_auc на тесте
# Вывести текстом и на графике =)

## Your Code Here

### Больше обязательных заданий нет, но ниже вы можете провести исследование, поиграться с данными
### Это возможность написать код, который я посмотрю и в случае чего откомметирую. ;)
### Это не оценивается и остается на ваше усмотрение. Просто дополнительная возможность

In [ ]:
### И далее ;)